## Scraping the YouTube links from Youtuber Oxhorn

The context we want to send the model comes from a YouTube video. Let's download the video and transcribe it using [OpenAI's Whisper](https://openai.com/research/whisper).

In [1]:
import requests
from bs4 import BeautifulSoup
from pytube import Playlist
import os
import tempfile
from pytube import YouTube
import whisper

In [2]:
# Function to get all video URLs from a YouTube playlist
def get_youtube_playlist_videos(playlist_url):
    # Create a Playlist object
    playlist = Playlist(playlist_url)
    
    # Extract all video URLs
    video_urls = [video.watch_url for video in playlist.videos]
    
    return video_urls

In [3]:
# URLs of the YouTube playlists
playlist_urls = [
    "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSUBHnhAvuNB_c4XXR-Oscl",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTZo9b8VjVb46S-JVMztEJw",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRU9Lv-cBja5O0sNTrayXiF",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQ9BFQIOjTGwTXXA3Vy7Tou",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRXwNRbEdFQwwYseR5QXwwO",
    "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQLzrm-usenWm_qO02YubJ8",     #fallout4 fullstory
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTGBh4cNGpkNm0dSXj_BOJw",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSI9XhqaOS1XZ8z_diMAgYj",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRpIbzPCj_nnYg6QTiLOjVC",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WS45ixut6N7eLuKj1PF__cA",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WT5u-cW4EDwUpAnpgGQdIxm",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQmcNS5xNuBSZs4e8YhyRDt",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WS1jw3zDYuz6M-G41oPMUfM",
    "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTIjWTdtBuSfPu5Z-K32vsZ",     #fallout2 fullstory
    "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRSCJe-Uegt2qyPcvjsSm-4",     #fallout3 full story
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQ36CjWBh-bYAga2Nswyzf5",   #wasteland survival guide
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTTtxrltd04qJ2bm-YaBcs-",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WR7S7WYNHxE--e67vWJrmSi",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRfqpytFWMP39R6BoFauU3K",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSTdI9WNBp4y1A9msSfO_Ey",   #fallout 76 lore
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WS-5Jc-_sb3KShgf9W4xeez",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTon2ViXxQG1C2I50PhAyO9",
    "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSzYFziFfkFzUO2E9bLF57x",  #fallout1
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRQA0VXnqj_4ltK5TrKjapT",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSrVdrnIoSqNfdBWAKICMe7",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRpTZu11YgQqAq93GvhRQHi",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WS_TdZuVcBcLUEFbt8kio0F",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WS4Yj_lS3Xy1llN8FuGio-5",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRvKbqFkWxbbQtwBFmUVCtm",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTnhwq240l-7wk0dMEULE20",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQUX5VkmGfaNwKuJ9Q3oI3N",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTwmMAZVO5bPIhFaOlTrpbw",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQf4GaBtckx0RCUFmNi9O2g",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WR74CFxfejPI5PhntgtH9OA",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQ7RJQJcKJUIN3seenEWYPv",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQYnn9qyAwwN0PJHJ9GkcJn",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRHv6pPBvniIWx3YLzK1IaE",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQwnCC-pDo28G3dPUBJ0i3u",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WToEHqy6t37v84bS_E2jN-0",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSrnD-Exn1AqGCUlmqmFmkX",
    "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRO3Jt_s-4Vo7LNGWkG1XeM", # story of fallout new vegas
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTBfoteH88GmTBWXuAJiipf",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSgS5o_tzQcj2BheViB_uJ4",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTwZgVVBq7mB25vvxtGuTAR",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRC1GXW0FQAtfXykyRWEUOD",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WR4KC92kl1uE72k05lD18vw",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRqgzWOcgsmEt8lvY141PLk",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WT--KQvjPOcdHNKuqrAvswo",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRYEUyzaHY7tlk3p7vZWJUV",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQc49EPSTyK4T4duqrAWDad",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSIiXwZkpaYLx2EiK-wH48d",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRMRfGDInfEPPkG1sCAtHTX",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSZyQQ1Qf1ohA_wUNE3g094",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSefsj5nA7zZGpxmuGPGUQr",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WSt0G3tpsp9qGisqfFaA0YL",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQZs-bB41frYA_ufAzM9pKP",   #Fallout3 lore
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRLK0X5D6uUcSWv6z44F-nN",  # fallout newvegas lore
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRCIEdKByKVIWzxLGowUrMa",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRPF2PS2ejkRRirMDiLDpGQ",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRxmWlhwuB89aQY9xJIv6Mf",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WS4SypmwvikKpJXeeCJyUcD"#,   # Fallout 4 lore
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WQVWndxsfmuhlXnIstMJvLs",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRhIVglPoRU-0gOGIc3XZhB",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRJrrgIhhY0Plo0KR-ZPZCE",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WTc3b4ZZJRUC3lKAX5P6ngC",
    # "https://www.youtube.com/playlist?list=PLX3Sax_xs4WRnYH255cFTbu9pespjhpkt"
]

# Get all video URLs from each playlist
all_video_urls = []

In [4]:
for playlist_url in playlist_urls:
    video_urls = get_youtube_playlist_videos(playlist_url)
    all_video_urls.extend(video_urls)

# Remove duplicates by converting the list to a set and then back to a list
unique_video_urls = list(set(all_video_urls))

# Print the unique video URLs
for url in unique_video_urls:
    print(url)

https://youtube.com/watch?v=lk2CH1nn8bE
https://youtube.com/watch?v=8vI_jVn2Cyc
https://youtube.com/watch?v=5hEPKp6Gd0o
https://youtube.com/watch?v=HSbShnzuqYQ
https://youtube.com/watch?v=xot6j-NakUM
https://youtube.com/watch?v=-K6MQbe71UQ
https://youtube.com/watch?v=hareqsme9kc
https://youtube.com/watch?v=njdky0a-vYw
https://youtube.com/watch?v=xW-7S-BvGHw
https://youtube.com/watch?v=xmccrzvJcxc
https://youtube.com/watch?v=6U6BxubiE0A
https://youtube.com/watch?v=Bs85HF1mKns
https://youtube.com/watch?v=jd9jtpvXvUQ
https://youtube.com/watch?v=gzRM2ubf5Fw
https://youtube.com/watch?v=dHAwrVN70BY
https://youtube.com/watch?v=FYZ3zzEoIzQ
https://youtube.com/watch?v=LM5c3KRS-84
https://youtube.com/watch?v=sbGqQnrBKK8
https://youtube.com/watch?v=tVdo9MLatps
https://youtube.com/watch?v=3bsVFF2Q7oo
https://youtube.com/watch?v=ViXfryqNN58
https://youtube.com/watch?v=ZSMSyM-Nhn4
https://youtube.com/watch?v=kcUJjFigpkY
https://youtube.com/watch?v=x7nkWMrOeZw
https://youtube.com/watch?v=4G9TMzzCOfg


In [5]:
len(all_video_urls)

170

In [6]:
len(unique_video_urls)

170

## Transcribing the YouTube Video

The context we want to send the model comes from a YouTube video. Let's download the video and transcribe it using [OpenAI's Whisper](https://openai.com/research/whisper).

In [7]:
# Ensure the directory exists
output_dir = "./YouTube_oxhorn"
os.makedirs(output_dir, exist_ok=True)

def get_video_metadata(video_url):
    yt = YouTube(video_url)
    metadata = {
        'title': yt.title,
        'views': yt.views,
        'length': yt.length,
        'author': yt.author,
        'publish_date': yt.publish_date,
        'description': yt.description
    }
    return metadata

def transcribe_video_audio(video_url, output_dir):
    yt = YouTube(video_url)
    audio = yt.streams.filter(only_audio=True).first()
    
    # Load Whisper model
    whisper_model = whisper.load_model("base")
    
    with tempfile.TemporaryDirectory() as tmpdir:
        file_path = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file_path, fp16=False)["text"].strip()
        
        # Create a transcription file named after the video ID in the output directory
        video_id = yt.video_id
        transcription_filename = os.path.join(output_dir, f"transcription_{video_id}.txt")
        
        with open(transcription_filename, "w") as file:
            file.write(transcription)
        
        print(f"Transcription for video '{yt.title}' saved to {transcription_filename}")

In [8]:
# Process each video URL
for url in unique_video_urls:
    transcribe_video_audio(url, output_dir)

Transcription for video 'The Story of Fallout 2 Part 2: Klamath Falls and Sulik' saved to ./YouTube_oxhorn/transcription_lk2CH1nn8bE.txt
Transcription for video 'The Warwick Initiative - The Story of Fallout 4 Part 21' saved to ./YouTube_oxhorn/transcription_8vI_jVn2Cyc.txt
Transcription for video 'The Great Green Jewel - The Full Story of Fallout 4 Part 4' saved to ./YouTube_oxhorn/transcription_5hEPKp6Gd0o.txt
Transcription for video 'The Brotherhood of Steel in San Francisco & ACE the Lonely Computer - The Story of Fallout 2 Part 31' saved to ./YouTube_oxhorn/transcription_HSbShnzuqYQ.txt
Transcription for video 'The Secrets of the Institute - The Story of Fallout 4 Part 17' saved to ./YouTube_oxhorn/transcription_xot6j-NakUM.txt
Transcription for video 'Trapped in Cambridge Polymer Labs - The Story of Fallout 4 Part 33' saved to ./YouTube_oxhorn/transcription_-K6MQbe71UQ.txt
Transcription for video 'How The Railroad Destroys The Brotherhood - The Story of Fallout 4 Part 43' saved t

AgeRestrictedError: p4hdqC9-KAg is age restricted, and can't be accessed without logging in.

In [13]:
unique_video_urls = unique_video_urls[38:]

In [ ]:
for url in unique_video_urls:
    transcribe_video_audio(url, output_dir)

Transcription for video 'Norm: A Profile in Courage - The Fallout Show' saved to ./YouTube_spanish/transcription_MzqOGw3X_n8.txt
Transcription for video 'The Story of Fallout 2 Part 4: Modoc, the Ghost Farm, & a Big Chicken - Fallout 2 Lore' saved to ./YouTube_spanish/transcription_5pS_nD1WAmg.txt
Transcription for video 'Broken Steel 7: Shock Value - Unearthing a Tesla Coil from the Old Olney Powerworks - Fallout 3 Lore' saved to ./YouTube_spanish/transcription_Pj5dotgf9nA.txt
Transcription for video 'Virgil's Lab in the Glowing Sea - The Full Story of Fallout 4 Part 9' saved to ./YouTube_spanish/transcription_BxIKAssx0Is.txt
Transcription for video 'Powering Up - The Story of Fallout 4 Part 28' saved to ./YouTube_spanish/transcription_B8dOLe15LbY.txt
Transcription for video 'The Story of Fallout 3 Part 17: The Sacrifice - All Possible Endings' saved to ./YouTube_spanish/transcription_F3qi2niufe4.txt
Transcription for video 'The Full Story of Fallout 1 Part 2: Vault 15 - An Experiment

Let's read the transcription and display the first few characters to ensure everything works as expected.

For videos in spanish

In [9]:
videos_in_spanish = [
    "https://youtu.be/2mpMyjvN8M4?si=YElDhsaQC6cFpLN9",
    "https://youtu.be/CZ732Ssn-Cw?si=BJTbzGzmQGhL-yTH",
    "https://youtu.be/W4cBLfI1oWY?si=kvyEXH_0mO9JKTec",
    "https://youtu.be/iNfeCytVRsc?si=U4Rx5JEW8I7VLb7w"
]

In [10]:
# Ensure the directory exists
output_dir = "./YouTube_spanish"
os.makedirs(output_dir, exist_ok=True)

In [11]:
# Process each video URL
for url in videos_in_spanish:
    transcribe_video_audio(url, output_dir)

Transcription for video 'FALLOUT LORE: MÉXICO / UNIVERSO FALLOUT 🇲🇽' saved to ./YouTube_spanish/transcription_2mpMyjvN8M4.txt
Transcription for video '¿Qué pasó con Latinoamérica en el Universo de Fallout?' saved to ./YouTube_spanish/transcription_CZ732Ssn-Cw.txt
Transcription for video 'Fallout | MEXICO' saved to ./YouTube_spanish/transcription_W4cBLfI1oWY.txt
Transcription for video 'Fallout Lore: Mexico (and Raul)' saved to ./YouTube_spanish/transcription_iNfeCytVRsc.txt


In [15]:
from translate import Translator
# Process each transcription file and translate its content

translator = Translator(to_lang="en")

for filename in os.listdir(output_dir):
    if filename.startswith("transcription"):
        file_path = os.path.join(output_dir, filename)
        
        with open(file_path, "r") as file:
            text = file.read()
            
            # Translate text
            translated_text = translator.translate(text)
            
            # Write translated text back to the original file
            with open(file_path, "w") as file:
                file.write(translated_text)

print("Translated texts have been saved back to the original files.")

Translated texts have been saved back to the original files.
